# MVP – Engenharia de Dados (PUC-Rio)
## Pipeline em Databricks (SQL)

Fonte: Kaggle — Brazilian Stock Market (CSV)


In [0]:
-- Criação do schema Bronze
CREATE SCHEMA IF NOT EXISTS bronze;

## Dados brutos (Bronze)

### Views Bronze – Ações B3


In [0]:
CREATE OR REPLACE VIEW bronze.acoes_b3 AS
SELECT *
FROM read_files(
  "dbfs:/Volumes/workspace/default/raw/acoes_b3.csv",
  format => "csv",
  header => "true",
  inferSchema => "true"
);


### Views Bronze – Indicadores Econômicos

In [0]:
CREATE OR REPLACE VIEW bronze.indicadores_economicos AS
SELECT *
FROM read_files(
  "dbfs:/Volumes/workspace/default/raw/indicadores_economicos.csv",
  format => "csv",
  header => "true",
  inferSchema => "true"
);


### Views Bronze – Índices B3

In [0]:
CREATE OR REPLACE VIEW bronze.indices_b3 AS
SELECT *
FROM read_files(
  "dbfs:/Volumes/workspace/default/raw/indices_b3.csv",
  format => "csv",
  header => "true",
  inferSchema => "true"
);

## Exploração inicial dos dados

### Amostra: Tabelas Bronze

In [0]:
SELECT * FROM bronze.acoes_b3 LIMIT 5;

Date,Symbol,Adj Close,Close,High,Low,Open,Volume,_rescued_data
2022-10-03T00:00:00.000Z,SMAB11,8.359999656677246,8.359999656677246,8.369999885559082,7.96999979019165,8.0,579.0,null
2010-01-04T00:00:00.000Z,MMXM3,1599.1749267578125,1599.1749267578125,1599.1749267578125,1555.1378173828125,1556.39599609375,20984.0,null
2010-01-05T00:00:00.000Z,MMXM3,1609.240478515625,1609.240478515625,1616.789794921875,1567.7198486328125,1600.43310546875,22586.0,null
2010-01-06T00:00:00.000Z,MMXM3,1680.9580078125,1680.9580078125,1685.9908447265625,1599.1749267578125,1616.789794921875,37998.0,null
2010-01-07T00:00:00.000Z,MMXM3,1698.5728759765625,1698.5728759765625,1704.8638916015625,1641.9537353515625,1660.8267822265625,33651.0,null


In [0]:
SELECT * FROM bronze.indicadores_economicos LIMIT 5;

Date,Taxa Selic,IPCA,IGP-M,INPC,Desemprego PNADC,_rescued_data
2010-01-01,8.75,0.75,0.63,0.88,null,null
2010-01-02,8.75,null,null,null,null,null
2010-01-03,8.75,null,null,null,null,null
2010-01-04,8.75,null,null,null,null,null
2010-01-05,8.75,null,null,null,null,null


In [0]:
SELECT * FROM bronze.indices_b3 LIMIT 5;

Date,Symbol,Adj Close,Close,High,Low,Open,Volume,_rescued_data


**Observação:**  
A tabela `indices_b3` não apresentou registros após a leitura dos dados brutos (row_count = 0).  
O conjunto foi mantido para documentação e possível uso futuro, mas não será utilizado nas análises deste MVP

In [0]:
SELECT 'acoes_b3' AS tabela, COUNT(*) AS row_count FROM bronze.acoes_b3
UNION ALL
SELECT 'indicadores_economicos' AS tabela, COUNT(*) AS row_count FROM bronze.indicadores_economicos
UNION ALL
SELECT 'indices_b3' AS tabela, COUNT(*) AS row_count FROM bronze.indices_b3;


tabela,row_count
acoes_b3,1031282
indicadores_economicos,5537
indices_b3,0


### Estrutura e Tipos de dados

In [0]:
-- Ações B3
DESCRIBE bronze.acoes_b3;

col_name,data_type,comment
Date,timestamp,null
Symbol,string,null
Adj Close,double,null
Close,double,null
High,double,null
Low,double,null
Open,double,null
Volume,double,null
_rescued_data,string,null


In [0]:
-- Indicadores Econômicos
DESCRIBE bronze.indicadores_economicos;

col_name,data_type,comment
Date,date,null
Taxa Selic,double,null
IPCA,double,null
IGP-M,double,null
INPC,double,null
Desemprego PNADC,double,null
_rescued_data,string,null


### Estatísticas Descritivas Iniciais – Ações da B3

In [0]:
-- Estatísticas descritivas iniciais – Ações B3
-- Valores numéricos limitados a 2 casas decimais

SELECT
  'nulos' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  CAST(SUM(CASE WHEN `Adj Close` IS NULL THEN 1 ELSE 0 END) AS DOUBLE) AS `Adj Close`,
  CAST(SUM(CASE WHEN Close IS NULL THEN 1 ELSE 0 END) AS DOUBLE)       AS Close,
  CAST(SUM(CASE WHEN High IS NULL THEN 1 ELSE 0 END) AS DOUBLE)        AS High,
  CAST(SUM(CASE WHEN Low  IS NULL THEN 1 ELSE 0 END) AS DOUBLE)        AS Low,
  CAST(SUM(CASE WHEN Open IS NULL THEN 1 ELSE 0 END) AS DOUBLE)        AS Open,
  CAST(SUM(CASE WHEN Volume IS NULL THEN 1 ELSE 0 END) AS DOUBLE)      AS Volume
FROM bronze.acoes_b3

UNION ALL

SELECT
  'min' AS metrica,
  MIN(Date) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  ROUND(MIN(`Adj Close`), 2),
  ROUND(MIN(Close), 2),
  ROUND(MIN(High), 2),
  ROUND(MIN(Low), 2),
  ROUND(MIN(Open), 2),
  ROUND(MIN(Volume), 2)
FROM bronze.acoes_b3

UNION ALL

SELECT
  'max' AS metrica,
  MAX(Date) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  ROUND(MAX(`Adj Close`), 2),
  ROUND(MAX(Close), 2),
  ROUND(MAX(High), 2),
  ROUND(MAX(Low), 2),
  ROUND(MAX(Open), 2),
  ROUND(MAX(Volume), 2)
FROM bronze.acoes_b3

UNION ALL

SELECT
  'media' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  ROUND(AVG(`Adj Close`), 2),
  ROUND(AVG(Close), 2),
  ROUND(AVG(High), 2),
  ROUND(AVG(Low), 2),
  ROUND(AVG(Open), 2),
  ROUND(AVG(Volume), 2)
FROM bronze.acoes_b3

UNION ALL

SELECT
  'desvio_padrao' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  ROUND(stddev_samp(`Adj Close`), 2),
  ROUND(stddev_samp(Close), 2),
  ROUND(stddev_samp(High), 2),
  ROUND(stddev_samp(Low), 2),
  ROUND(stddev_samp(Open), 2),
  ROUND(stddev_samp(Volume), 2)
FROM bronze.acoes_b3

UNION ALL

SELECT
  'mediana' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  ROUND(percentile_approx(`Adj Close`, 0.5), 2),
  ROUND(percentile_approx(Close, 0.5), 2),
  ROUND(percentile_approx(High, 0.5), 2),
  ROUND(percentile_approx(Low, 0.5), 2),
  ROUND(percentile_approx(Open, 0.5), 2),
  ROUND(percentile_approx(Volume, 0.5), 2)
FROM bronze.acoes_b3;



metrica,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
nulos,null,null,0.0,0.0,0.0,0.0,0.0,0.0
min,2010-01-04T00:00:00.000Z,null,-50.84,0.0,0.0,0.0,0.0,0.0
max,2025-02-17T00:00:00.000Z,null,44594.76,49200.49,49500.5,45600.46,49200.49,6.98950612E8
media,null,null,61.95,71.13,70.55,69.22,69.89,1883205.24
desvio_padrao,null,null,455.41,569.24,573.31,563.2,568.24,7696832.13
mediana,null,null,13.6,16.67,16.22,15.81,16.0,13800.0


### Checagem rápida de zeros e negativos – Ações B3

In [0]:
SELECT
  'zeros' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  CAST(SUM(CASE WHEN `Adj Close` = 0 THEN 1 ELSE 0 END) AS DOUBLE) AS `Adj Close`,
  CAST(SUM(CASE WHEN Close = 0 THEN 1 ELSE 0 END) AS DOUBLE)      AS Close,
  CAST(SUM(CASE WHEN High = 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS High,
  CAST(SUM(CASE WHEN Low  = 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS Low,
  CAST(SUM(CASE WHEN Open = 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS Open,
  CAST(SUM(CASE WHEN Volume = 0 THEN 1 ELSE 0 END) AS DOUBLE)     AS Volume
FROM bronze.acoes_b3

UNION ALL

SELECT
  'negativos' AS metrica,
  CAST(NULL AS TIMESTAMP) AS Date,
  CAST(NULL AS STRING) AS Symbol,
  CAST(SUM(CASE WHEN `Adj Close` < 0 THEN 1 ELSE 0 END) AS DOUBLE) AS `Adj Close`,
  CAST(SUM(CASE WHEN Close < 0 THEN 1 ELSE 0 END) AS DOUBLE)      AS Close,
  CAST(SUM(CASE WHEN High < 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS High,
  CAST(SUM(CASE WHEN Low  < 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS Low,
  CAST(SUM(CASE WHEN Open < 0 THEN 1 ELSE 0 END) AS DOUBLE)       AS Open,
  CAST(SUM(CASE WHEN Volume < 0 THEN 1 ELSE 0 END) AS DOUBLE)     AS Volume
FROM bronze.acoes_b3;


metrica,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
zeros,null,null,1.0,1.0,26721.0,26720.0,26987.0,287767.0
negativos,null,null,10303.0,0.0,0.0,0.0,0.0,0.0


#### Síntese Ações B3
A análise exploratória inicial identificou a presença de valores zero em colunas de preço (Open, High, Low e Close), indicando provável uso de placeholders no conjunto de dados bruto.
Também foram encontrados valores negativos na coluna Adj Close, sugerindo inconsistências associadas a ajustes corporativos ou problemas na fonte de dados.
Esses pontos serão considerados nas etapas posteriores de tratamento e modelagem dos dados.

### Estatísticas descritivas iniciais - Indicadores Econômicos

In [0]:
SELECT
  'nulos' AS metrica,
  CAST(NULL AS DATE) AS Date,
  ROUND(CAST(SUM(CASE WHEN `Taxa Selic` IS NULL THEN 1 ELSE 0 END) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(SUM(CASE WHEN IPCA IS NULL THEN 1 ELSE 0 END) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(SUM(CASE WHEN `IGP-M` IS NULL THEN 1 ELSE 0 END) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(SUM(CASE WHEN INPC IS NULL THEN 1 ELSE 0 END) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(SUM(CASE WHEN `Desemprego PNADC` IS NULL THEN 1 ELSE 0 END) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos

UNION ALL
SELECT
  'min' AS metrica,
  MIN(Date) AS Date,
  ROUND(CAST(MIN(`Taxa Selic`) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(MIN(IPCA) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(MIN(`IGP-M`) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(MIN(INPC) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(MIN(`Desemprego PNADC`) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos

UNION ALL
SELECT
  'max' AS metrica,
  MAX(Date) AS Date,
  ROUND(CAST(MAX(`Taxa Selic`) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(MAX(IPCA) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(MAX(`IGP-M`) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(MAX(INPC) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(MAX(`Desemprego PNADC`) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos

UNION ALL
SELECT
  'media' AS metrica,
  CAST(NULL AS DATE) AS Date,
  ROUND(CAST(AVG(`Taxa Selic`) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(AVG(IPCA) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(AVG(`IGP-M`) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(AVG(INPC) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(AVG(`Desemprego PNADC`) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos

UNION ALL
SELECT
  'desvio_padrao' AS metrica,
  CAST(NULL AS DATE) AS Date,
  ROUND(CAST(STDDEV_SAMP(`Taxa Selic`) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(STDDEV_SAMP(IPCA) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(STDDEV_SAMP(`IGP-M`) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(STDDEV_SAMP(INPC) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(STDDEV_SAMP(`Desemprego PNADC`) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos

UNION ALL
SELECT
  'mediana' AS metrica,
  CAST(NULL AS DATE) AS Date,
  ROUND(CAST(percentile_approx(`Taxa Selic`, 0.5) AS DOUBLE), 2) AS `Taxa Selic`,
  ROUND(CAST(percentile_approx(IPCA, 0.5) AS DOUBLE), 2) AS IPCA,
  ROUND(CAST(percentile_approx(`IGP-M`, 0.5) AS DOUBLE), 2) AS `IGP-M`,
  ROUND(CAST(percentile_approx(INPC, 0.5) AS DOUBLE), 2) AS INPC,
  ROUND(CAST(percentile_approx(`Desemprego PNADC`, 0.5) AS DOUBLE), 2) AS `Desemprego PNADC`
FROM bronze.indicadores_economicos;


metrica,Date,Taxa Selic,IPCA,IGP-M,INPC,Desemprego PNADC
nulos,null,0.0,5356.0,5356.0,5356.0,5383.0
min,2010-01-01,2.0,-0.68,-1.93,-0.6,6.1
max,2025-02-27,14.25,1.62,4.34,1.71,14.9
media,null,9.64,0.48,0.61,0.47,10.05
desvio_padrao,null,3.46,0.36,0.9,0.39,2.66
mediana,null,10.5,0.45,0.57,0.46,9.3


### Linhagem dos Dados

Esta seção documentaa origem e o fluxo dos dados utilizados neste notebook.

- Os dados são carregados a partir da camada Bronze, originada de arquivos CSV públicos.
- As tabelas Bronze são utilizadas como fonte para a construção das tabelas analíticas na camada Gold.
- As transformações e cargas realizadas neste notebook seguem o modelo lógico definido no README do projeto.


## Camada Gold – Implementação do Pipeline de ETL

Nesta etapa, foi implementado o pipeline de Transformação e Carga (ETL) para a camada Gold, seguindo o modelo de dados em Esquema Estrela definido anteriormente.

A camada Gold concentra os dados já tratados, integrados e prontos para análise, garantindo:
- padronização de chaves;
- integridade referencial;
- consistência temporal entre os conjuntos de dados.



In [0]:
CREATE SCHEMA IF NOT EXISTS gold;

### dim_date

In [0]:
CREATE OR REPLACE TABLE gold.dim_date AS
SELECT
  ROW_NUMBER() OVER (ORDER BY d) AS date_id,
  d AS date,
  YEAR(d)  AS year,
  MONTH(d) AS month,
  DAY(d)   AS day
FROM (
  SELECT DISTINCT CAST(Date AS DATE) AS d FROM bronze.acoes_b3
  UNION
  SELECT DISTINCT CAST(Date AS DATE) AS d FROM bronze.indicadores_economicos
) dates
WHERE d BETWEEN DATE('2010-01-04') AND DATE('2025-02-17');



num_affected_rows,num_inserted_rows


### dim_acoes_b3

In [0]:
CREATE OR REPLACE TABLE gold.dim_acoes_b3 AS
SELECT
  ROW_NUMBER() OVER (ORDER BY symbol) AS symbol_id,
  symbol
FROM (
  SELECT DISTINCT Symbol AS symbol
  FROM bronze.acoes_b3
  WHERE Symbol IS NOT NULL
);


num_affected_rows,num_inserted_rows


## Criação das Tabelas Fato

Foram criadas duas tabelas fato na camada Gold, representando métricas quantitativas de negócio:

- `gold.fact_acoes_b3`
- `gold.fact_indicadores_economicos`

Ambas utilizam a dimensão temporal (`dim_date`) como referência comum, permitindo análises comparativas ao longo do tempo.


### fact_acoes_b3

A tabela `fact_acoes_b3` armazena os preços históricos e volumes negociados das ações listadas na B3.

Durante a carga:
- os dados foram filtrados para o intervalo comum entre os datasets;
- a chave temporal (`date_id`) foi obtida por meio de junção com a dimensão `dim_date`;
- a chave do ativo (`symbol_id`) referencia a dimensão `dim_acoes_b3`.

In [0]:
CREATE OR REPLACE TABLE gold.fact_acoes_b3 AS
SELECT
  d.date_id,
  a.symbol_id,
  b.`Adj Close` AS adj_close,
  b.Close       AS close,
  b.High        AS high,
  b.Low         AS low,
  b.Open        AS open,
  b.Volume      AS volume,
  b._rescued_data
FROM bronze.acoes_b3 b
JOIN gold.dim_date d
  ON CAST(b.Date AS DATE) = d.date
JOIN gold.dim_acoes_b3 a
  ON b.Symbol = a.symbol
WHERE CAST(b.Date AS DATE)
      BETWEEN DATE('2010-01-04') AND DATE('2025-02-17');


num_affected_rows,num_inserted_rows


### fact_indicadores_economicos

A tabela `fact_indicadores_economicos` armazena indicadores macroeconômicos utilizados como contexto para análise dos ativos financeiros.

Foram incluídos os seguintes indicadores:
- taxa Selic;
- IPCA;
- IGP-M;
- INPC;
- desemprego (PNADC).

A carga foi realizada garantindo alinhamento temporal com a dimensão `dim_date`, utilizando o mesmo intervalo de datas adotado para a tabela de ações.


In [0]:
CREATE OR REPLACE TABLE gold.fact_indicadores_economicos AS
SELECT
  d.date_id,
  i.`Taxa Selic`        AS taxa_selic,
  i.IPCA                AS ipca,
  i.`IGP-M`             AS igpm,
  i.INPC                AS inpc,
  i.`Desemprego PNADC`  AS desemprego_pnadc,
  i._rescued_data
FROM bronze.indicadores_economicos i
JOIN gold.dim_date d
  ON CAST(i.Date AS DATE) = d.date
WHERE CAST(i.Date AS DATE)
      BETWEEN DATE('2010-01-04') AND DATE('2025-02-17');


num_affected_rows,num_inserted_rows


### Amostra: Tabelas Gold

In [0]:
SELECT * FROM gold.dim_date LIMIT 5;

date_id,date,year,month,day
1,2010-01-04,2010,1,4
2,2010-01-05,2010,1,5
3,2010-01-06,2010,1,6
4,2010-01-07,2010,1,7
5,2010-01-08,2010,1,8


In [0]:
SELECT * FROM gold.dim_acoes_b3 LIMIT 5;

symbol_id,symbol
1,A2MC34
2,A2VL34
3,AALR3
4,AAPL34
5,ABTT34


In [0]:
SELECT * FROM gold.fact_acoes_b3 LIMIT 5

date_id,symbol_id,adj_close,close,high,low,open,volume,_rescued_data
4574,404,25.0,25.0,25.39999961853028,25.0,25.0,1500.0,null
4575,404,25.0,25.0,25.290000915527344,24.75,24.75,3500.0,null
4576,404,25.100000381469727,25.100000381469727,25.100000381469727,25.059999465942383,25.059999465942383,400.0,null
4579,404,25.0,25.0,25.15999984741211,25.0,25.15999984741211,700.0,null
4580,404,25.0,25.0,25.06999969482422,24.81999969482422,25.0,1900.0,null


In [0]:
SELECT * FROM gold.fact_indicadores_economicos LIMIT 5

date_id,taxa_selic,ipca,igpm,inpc,desemprego_pnadc,_rescued_data
1,8.75,null,null,null,null,null
2,8.75,null,null,null,null,null
3,8.75,null,null,null,null,null
4,8.75,null,null,null,null,null
5,8.75,null,null,null,null,null


In [0]:
SELECT
  MIN(d.date) AS data_minima,
  MAX(d.date) AS data_maxima
FROM gold.fact_acoes_b3 f
JOIN gold.dim_date d ON f.date_id = d.date_id;


data_minima,data_maxima
2010-01-04,2025-02-17


## Padronização do Intervalo Temporal

Para garantir consistência analítica entre as tabelas fato, foi adotado um intervalo temporal comum, presente em ambos os datasets:

- **Data mínima:** 2010-01-04  
- **Data máxima:** 2025-02-17

Esse filtro assegura que comparações entre indicadores econômicos e desempenho das ações sejam realizadas apenas em períodos com dados disponíveis em ambas as fontes.


## Validação da Persistência e Volume dos Dados

Após a criação das tabelas na camada Gold, foram realizadas validações básicas de persistência e volume de registros, confirmando que os dados foram carregados corretamente.


In [0]:
SHOW TABLES IN gold;

database,tableName,isTemporary
gold,dim_acoes_b3,false
gold,dim_date,false
gold,fact_acoes_b3,false
gold,fact_indicadores_economicos,false


In [0]:
DESCRIBE TABLE gold.fact_acoes_b3;

col_name,data_type,comment
date_id,int,null
symbol_id,int,null
adj_close,double,null
close,double,null
high,double,null
low,double,null
open,double,null
volume,double,null
_rescued_data,string,null


In [0]:
DESCRIBE TABLE gold.fact_indicadores_economicos;

col_name,data_type,comment
date_id,int,null
taxa_selic,double,null
ipca,double,null
igpm,double,null
inpc,double,null
desemprego_pnadc,double,null
_rescued_data,string,null


In [0]:
SELECT COUNT(*) FROM gold.fact_acoes_b3;


COUNT(*)
1031282


In [0]:
SELECT COUNT(*) FROM gold.fact_indicadores_economicos;

COUNT(*)
5524


Os resultados confirmam que:
- as tabelas foram persistidas corretamente na camada Gold;
- a estrutura está aderente ao modelo lógico definido;
- o volume de registros é compatível com os dados de origem tratados nas etapas anteriores.


## Análise de Qualidade dos Dados – Camada Gold
Nesta etapa foi realizada a análise de qualidade dos dados persistidos na camada Gold, com o objetivo de verificar valores ausentes, inconsistências, conformidade com os domínios definidos no catálogo de dados e coerência temporal dos registros.
As verificações consideram que parte das inconsistências identificadas na camada Bronze (valores zero e negativos) foram preservadas propositalmente na Gold para fins de análise e transparência do MVP.


#### Verificação de valores nulos - fact_acoes_b3

In [0]:
SELECT
  COUNT(*) AS total_registros,
  SUM(CASE WHEN open IS NULL THEN 1 ELSE 0 END) AS open_nulls,
  SUM(CASE WHEN close IS NULL THEN 1 ELSE 0 END) AS close_nulls,
  SUM(CASE WHEN high IS NULL THEN 1 ELSE 0 END) AS high_nulls,
  SUM(CASE WHEN low IS NULL THEN 1 ELSE 0 END) AS low_nulls,
  SUM(CASE WHEN volume IS NULL THEN 1 ELSE 0 END) AS volume_nulls
FROM gold.fact_acoes_b3;


total_registros,open_nulls,close_nulls,high_nulls,low_nulls,volume_nulls
1031282,0,0,0,0,0


**Resultado:**  
Não foram identificados valores nulos nas colunas de preços (open, close, high, low) e volume da tabela fact_acoes_b3, indicando consistência estrutural após a carga dos dados na camada Gold.

#### Verificação de valores inválidos (preços <= 0)

In [0]:
SELECT
  SUM(CASE WHEN open <= 0 THEN 1 ELSE 0 END) AS open_invalidos,
  SUM(CASE WHEN close <= 0 THEN 1 ELSE 0 END) AS close_invalidos,
  SUM(CASE WHEN high <= 0 THEN 1 ELSE 0 END) AS high_invalidos,
  SUM(CASE WHEN low <= 0 THEN 1 ELSE 0 END) AS low_invalidos
FROM gold.fact_acoes_b3;


open_invalidos,close_invalidos,high_invalidos,low_invalidos
26987,1,26721,26720


**Observação:**  
Foram identificados registros com valores iguais a zero ou negativos nas colunas de preços (open, high e low), comportamento já observado na camada Bronze e mantido na camada Gold.
Para as análises propostas neste MVP, será priorizado o uso do preço de fechamento (close), uma vez que apresentou apenas um registro inválido em todo o conjunto de dados, reduzindo o impacto de valores inconsistentes

#### Verificação específica do adj_close

In [0]:
SELECT
  COUNT(*) AS total_registros,
  SUM(CASE WHEN adj_close IS NULL THEN 1 ELSE 0 END) AS adj_close_nulls
FROM gold.fact_acoes_b3;



total_registros,adj_close_nulls
1031282,0


**Observação:**  
A coluna adj_close não apresenta valores nulos na camada Gold.

#### Qualidade dos indicadores econômicos

In [0]:
SELECT
  SUM(CASE WHEN taxa_selic < 0 THEN 1 ELSE 0 END) AS selic_invalidos,
  SUM(CASE WHEN ipca < -50 OR ipca > 100 THEN 1 ELSE 0 END) AS ipca_invalidos,
  SUM(CASE WHEN igpm < -50 OR igpm > 100 THEN 1 ELSE 0 END) AS igpm_invalidos,
  SUM(CASE WHEN desemprego_pnadc < 0 OR desemprego_pnadc > 100 THEN 1 ELSE 0 END) AS desemprego_invalidos
FROM gold.fact_indicadores_economicos;


selic_invalidos,ipca_invalidos,igpm_invalidos,desemprego_invalidos
0,0,0,0


**Observação:**  
Os indicadores macroeconômicos apresentaram valores dentro dos domínios esperados definidos no catálogo de dados.

#### Coerência temporal

In [0]:
SELECT
  MIN(d.date) AS data_minima,
  MAX(d.date) AS data_maxima
FROM gold.fact_acoes_b3 f
JOIN gold.dim_date d ON f.date_id = d.date_id;


data_minima,data_maxima
2010-01-04,2025-02-17


**Observação:**  
A análise temporal confirmou que os dados da camada Gold abrangem o período de **2010-01-04 a 2025-02-17**, intervalo comum entre as tabelas fato, garantindo consistência temporal para análises comparativas.

#### 1 - Considerando os últimos 3 anos, quais são as 5 ações que apresentaram o maior retorno acumulado?

- Período analisado: últimos 36 meses, com base na data máxima disponível (2025-02-17);
- Ativos analisado: ações negociadas na B3 (`fact_acoes_b3`);
- Métrica utilizada: retorno acumulado no período;
- Preço inicial e preço final expressos em reais (R$);
- Retorno em múltiplo (X): indica quantas vezes o preço final superou o inicial;
- Retorno percentual (%): representa a valorização total do ativo no período.

In [0]:
WITH periodo AS (
  SELECT
    date_id,
    date
  FROM gold.dim_date
  WHERE date BETWEEN add_months(DATE('2025-02-17'), -36)
                  AND DATE('2025-02-17')
),

precos_ordenados AS (
  SELECT
    a.symbol AS ticker,
    d.date   AS data,
    f.adj_close,
    ROW_NUMBER() OVER (
      PARTITION BY a.symbol ORDER BY d.date ASC
    ) AS rn_inicio,
    ROW_NUMBER() OVER (
      PARTITION BY a.symbol ORDER BY d.date DESC
    ) AS rn_fim
  FROM gold.fact_acoes_b3 f
  JOIN gold.dim_date d
    ON f.date_id = d.date_id
  JOIN gold.dim_acoes_b3 a
    ON f.symbol_id = a.symbol_id
  JOIN periodo p
    ON f.date_id = p.date_id
  WHERE f.adj_close IS NOT NULL
),

precos_inicio_fim AS (
  SELECT
    ticker,
    MAX(CASE WHEN rn_inicio = 1 THEN adj_close END) AS preco_inicial_num,
    MAX(CASE WHEN rn_fim = 1 THEN adj_close END)    AS preco_final_num
  FROM precos_ordenados
  GROUP BY ticker
),

retornos AS (
  SELECT
    ticker,
    preco_inicial_num,
    preco_final_num,
    (preco_final_num / preco_inicial_num)           AS retorno_multiplo_num,
    ((preco_final_num / preco_inicial_num) - 1) * 100 AS retorno_percentual_num
  FROM precos_inicio_fim
  WHERE preco_inicial_num > 0
)

SELECT
  ticker,
  CONCAT('R$ ', FORMAT_NUMBER(preco_inicial_num, 2)) AS preco_inicial,
  CONCAT('R$ ', FORMAT_NUMBER(preco_final_num, 2))   AS preco_final,
  CONCAT(FORMAT_NUMBER(retorno_multiplo_num, 2), 'x') AS retorno_multiplo,
  CONCAT(FORMAT_NUMBER(retorno_percentual_num, 2), '%') AS retorno_percentual
FROM retornos
ORDER BY retorno_multiplo_num DESC
LIMIT 5;


ticker,preco_inicial,preco_final,retorno_multiplo,retorno_percentual
BRPR3,R$ 3.78,R$ 131.17,34.73x,"3,372.59%"
IRBR3,R$ 3.29,R$ 56.85,17.28x,"1,627.96%"
NGRD3,R$ 2.77,R$ 21.05,7.60x,659.54%
OIBR4,R$ 1.53,R$ 8.33,5.44x,444.44%
M1TA34,R$ 38.34,R$ 148.46,3.87x,287.22%


#### 2. Considerando os últimos 3 anos, qual foi a média de retorno anual das ações? E como esse valor se comporta nos últimos 12 meses?

- Período analisado: últimos 12 meses, com base na data máxima disponível (2025-02-17);
- Ativos analisados: ações negociadas na B3 (`fact_acoes_b3`);
- Métrica utilizada: retorno acumulado em 12 meses;
- Retorno médio (%): representa a média dos retornos individuais das ações no período;
- Retorno mediano (%): indica o retorno central da distribuição, reduzindo o impacto de outliers;
- Quantidade de ações consideradas: total de ativos com dados válidos no período analisado.

In [0]:
WITH data_max_preco AS (
  SELECT
    MAX(d.date) AS data_max
  FROM gold.fact_acoes_b3 f
  JOIN gold.dim_date d
    ON f.date_id = d.date_id
  WHERE f.adj_close IS NOT NULL
),

precos_12m AS (
  SELECT
    a.symbol AS ticker,
    d.date,
    f.adj_close,
    ROW_NUMBER() OVER (PARTITION BY a.symbol ORDER BY d.date ASC)  AS rn_ini,
    ROW_NUMBER() OVER (PARTITION BY a.symbol ORDER BY d.date DESC) AS rn_fim
  FROM gold.fact_acoes_b3 f
  JOIN gold.dim_date d
    ON f.date_id = d.date_id
  JOIN gold.dim_acoes_b3 a
    ON f.symbol_id = a.symbol_id
  WHERE d.date BETWEEN add_months((SELECT data_max FROM data_max_preco), -12)
                   AND (SELECT data_max FROM data_max_preco)
    AND f.adj_close IS NOT NULL
),

precos_ini_fim AS (
  SELECT
    ticker,
    MAX(CASE WHEN rn_ini = 1 THEN adj_close END) AS preco_inicial,
    MAX(CASE WHEN rn_fim = 1 THEN adj_close END) AS preco_final
  FROM precos_12m
  GROUP BY ticker
),

retornos AS (
  SELECT
    ticker,
    (preco_final / preco_inicial) - 1 AS retorno_12m
  FROM precos_ini_fim
  WHERE preco_inicial IS NOT NULL
    AND preco_final IS NOT NULL
    AND preco_inicial > 0
)

SELECT
  CONCAT(
    FORMAT_NUMBER(AVG(retorno_12m) * 100, 2),
    '%'
  ) AS retorno_medio_12m_b3,

  CONCAT(
    FORMAT_NUMBER(PERCENTILE_APPROX(retorno_12m, 0.5) * 100, 2),
    '%'
  ) AS retorno_mediano_12m_b3,

  COUNT(*) AS qtd_acoes_consideradas
FROM retornos;


retorno_medio_12m_b3,retorno_mediano_12m_b3,qtd_acoes_consideradas
13.50%,-6.17%,432


#### 3 - Em qual ano a inflação medida pelo IPCA foi mais elevada e em qual ano foi mais baixa?

- Indicador analisado: IPCA (`fact_indicadores_economicos`);
- Tabela de datas utilizada: `dim_date`;
- Agregação: soma anual do IPCA (valores mensais somados por ano);
- Métrica utilizada: `ipca_anual_pct` (% acumulado no ano);
- A análise permite identificar os anos com maior e menor inflação anual no período disponível na base.

In [0]:
WITH macro_anual AS (
  SELECT
    YEAR(d.date) AS ano,
    ROUND(SUM(i.ipca), 2) AS ipca_anual_pct_num,
    ROUND(AVG(i.taxa_selic), 2) AS selic_media_anual_num,
    ROUND(AVG(i.desemprego_pnadc), 2) AS desemprego_medio_anual_num
  FROM gold.fact_indicadores_economicos i
  JOIN gold.dim_date d
    ON i.date_id = d.date_id
  WHERE i.ipca IS NOT NULL
    AND i.taxa_selic IS NOT NULL
    AND i.desemprego_pnadc IS NOT NULL
  GROUP BY YEAR(d.date)
)

SELECT
  ano,
  CONCAT(FORMAT_NUMBER(ipca_anual_pct_num, 2), '%') AS ipca_anual_pct,
  CONCAT(FORMAT_NUMBER(selic_media_anual_num, 2), '%') AS selic_media_anual,
  CONCAT(FORMAT_NUMBER(desemprego_medio_anual_num, 2), '%') AS desemprego_medio_anual
FROM macro_anual
ORDER BY desemprego_medio_anual_num DESC;


ano,ipca_anual_pct,selic_media_anual,desemprego_medio_anual
2020,4.44%,3.02%,13.50%
2021,9.63%,4.21%,13.49%
2017,2.91%,10.40%,12.88%
2018,3.69%,6.60%,12.38%
2019,4.23%,6.08%,12.06%
2016,6.12%,14.19%,11.38%
2022,5.66%,12.25%,9.51%
2015,10.19%,13.38%,8.42%
2023,4.53%,13.42%,8.04%
2012,4.68%,8.38%,7.38%


#### 4 - Qual é a volatilidade dos retornos das ações ao longo do tempo, destacando as 5 ações mais estáveis e as 5 mais voláteis?

- Período analisado: últimos 36 meses, com base na data máxima disponível;
- Ativos analisados: ações negociadas na B3 (`fact_acoes_b3`);
- Métrica utilizada: volatilidade dos retornos (desvio padrão);

In [0]:
WITH data_max AS (
  SELECT MAX(d.date) AS data_max
  FROM gold.fact_acoes_b3 f
  JOIN gold.dim_date d ON f.date_id = d.date_id
),

precos AS (
  SELECT
    a.symbol AS sigla,
    d.date,
    f.adj_close,
    LAG(f.adj_close) OVER (PARTITION BY a.symbol ORDER BY d.date) AS preco_anterior
  FROM gold.fact_acoes_b3 f
  JOIN gold.dim_date d ON f.date_id = d.date_id
  JOIN gold.dim_acoes_b3 a ON f.symbol_id = a.symbol_id
  WHERE d.date BETWEEN add_months((SELECT data_max FROM data_max), -36)
                   AND (SELECT data_max FROM data_max)
),

retornos AS (
  SELECT
    sigla,
    (adj_close / preco_anterior) - 1 AS retorno
  FROM precos
  WHERE preco_anterior IS NOT NULL
    AND preco_anterior > 0
),

volatilidade AS (
  SELECT
    sigla,
    STDDEV_SAMP(retorno) AS volatilidade
  FROM retornos
  GROUP BY sigla
),

rank_estaveis AS (
  SELECT
    sigla,
    ROUND(volatilidade * 100, 2) AS volatilidade_pct,
    ROW_NUMBER() OVER (ORDER BY volatilidade ASC) AS rn
  FROM volatilidade
  WHERE volatilidade IS NOT NULL
),

rank_volateis AS (
  SELECT
    sigla,
    ROUND(volatilidade * 100, 2) AS volatilidade_pct,
    ROW_NUMBER() OVER (ORDER BY volatilidade DESC) AS rn
  FROM volatilidade
  WHERE volatilidade IS NOT NULL
)

SELECT
  e.sigla AS sigla_estavel,
  CONCAT(FORMAT_NUMBER(e.volatilidade_pct, 2), '%') AS volatilidade_estavel_pct,
  v.sigla AS sigla_volatil,
  CONCAT(FORMAT_NUMBER(v.volatilidade_pct, 2), '%') AS volatilidade_volatil_pct
FROM (SELECT * FROM rank_estaveis WHERE rn <= 5) e
JOIN (SELECT * FROM rank_volateis WHERE rn <= 5) v
  ON e.rn = v.rn;


sigla_estavel,volatilidade_estavel_pct,sigla_volatil,volatilidade_volatil_pct
MMXM3,0.00%,IRBR3,245.59%
MMXM11,0.00%,NGRD3,113.02%
CEPE3,0.20%,BRPR3,106.80%
BPAR3,0.20%,AERI3,81.32%
COCE6,0.24%,SEQL3,80.30%


#### 5 -  Qual é a relação entre o índice de desemprego e a taxa Selic e IPCA ao longo do tempo?

- Período analisado: anos disponíveis na base de dados, com base na data máxima registrada;
- Indicadores analisados: IPCA, taxa Selic e desemprego (PNAD Contínua);
- Fonte dos dados: `fact_indicadores_economicos`;
- Métricas utilizadas:
- IPCA acumulado anual (%);
- taxa Selic média anual (%);
- desemprego médio anual (%);

In [0]:
WITH indicadores AS (
  SELECT
    YEAR(d.date) AS ano,
    i.taxa_selic,
    i.ipca,
    i.desemprego_pnadc
  FROM gold.fact_indicadores_economicos i
  JOIN gold.dim_date d
    ON i.date_id = d.date_id
  WHERE i.taxa_selic IS NOT NULL
    AND i.ipca IS NOT NULL
    AND i.desemprego_pnadc IS NOT NULL
)

SELECT
  ano,
  ROUND(CORR(taxa_selic, desemprego_pnadc), 3) AS corr_selic_desemprego,
  ROUND(CORR(ipca, desemprego_pnadc), 3)       AS corr_ipca_desemprego,
  COUNT(*)                                     AS qtd_registros
FROM indicadores
GROUP BY ano
ORDER BY ano;


ano,corr_selic_desemprego,corr_ipca_desemprego,qtd_registros
2012,0.954,-0.659,10
2013,-0.94,-0.329,12
2014,0.124,-0.015,12
2015,0.968,-0.585,12
2016,-0.432,-0.901,12
2017,0.771,-0.342,12
2018,0.297,0.19,12
2019,0.889,-0.177,12
2020,-0.967,0.548,12
2021,-0.977,-0.475,12
